# Isochrones for public parking lots in Tel Aviv

Created by [Dan Fishgold](https://dan.city)

Based on [Elad's LRT network isochrones](https://github.com/elad661/metroTLV_walkshed)

In [9]:
# Basic setup
import json
from tqdm.notebook import tqdm
import geopandas as gpd
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
import shapely.geometry
import sklearn
import re

ox.settings.log_console = True
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [4]:
# configure basic parameters
 # (Download the entire metropolitian area, and then some. Too bad OSM doesn't have a relationship for the Tel Aviv Metropoliain Area)
place = ["Tel Aviv District, Israel", "Center District, Israel"]
network_type = "walk"
trip_times = [5, 7, 10]  # in minutes
travel_speed = 4.5  # very approximate walking speed in km/hour (real humans might walk slower or faster)

## Download and prep the street network

In [5]:
# download the street network
graph = ox.graph_from_place(place, network_type=network_type)

In [6]:
# add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
for _, _, _, data in graph.edges(data=True, keys=True):
    data["time"] = data["length"] / meters_per_minute

In [7]:
projected_graph = ox.project_graph(graph)

## Load the parking lot locations

From [Tel Aviv's GIS server](https://gisn.tel-aviv.gov.il/arcgis/rest/services/IView2/MapServer/556)

In [33]:

# crs = projected_graph.graph['crs']
# gdf = gpd.read_file('https://gisn.tel-aviv.gov.il/arcgis/rest/services/IView2/MapServer/556/query?where=1%3D1&outFields=*&f=json').to_crs(crs)
# gdf = gdf[gdf['achuzot'] == 'כן']

## Alternatively: from Ahuzot Hahof

In [10]:
crs = projected_graph.graph['crs']
gdf = gpd.read_file("https://www.ahuzot.co.il/map/ParkingMap.aspx?gx=1234", driver='KML').to_crs(crs)
gdf['id'] = gdf.apply(lambda lot: int(re.search(r'href="https://www.ahuzot.co.il/Parking/ParkingDetails/\?ID=(\d+)"', lot.Description).group(1)), axis=1)
gdf.set_index('id')
del gdf['Description']

In [12]:
gdf['nearest_node'] = gdf.apply(lambda lot: ox.distance.nearest_nodes(projected_graph, lot.geometry.x, lot.geometry.y), axis=1)

## Generate the isochrones

In [25]:
# This function makes the isochrones, will be reused later
def make_iso_polys(G, center_node, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = {}
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance="time")

        node_points = [Point((data["x"], data["y"])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({"id": list(subgraph.nodes)}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index("id")

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get("geometry", LineString([f, t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).unary_union

        # try to fill in surrounded areas so shapes will appear solid and
        # blocks without white space inside them
        if infill and hasattr(new_iso, 'exterior'):
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys[trip_time] = new_iso
    return isochrone_polys

def get_geojson_geometry(polygon):
    """Get geojson-compatible geometry, projected to a useful CRS"""
    geometry = ox.projection.project_geometry(polygon, crs=projected_graph.graph['crs'], to_latlong=True)[0]
    rounded_geometry = shapely.wkt.loads(shapely.wkt.dumps(geometry, rounding_precision=7))
    return shapely.geometry.mapping(rounded_geometry)



In [21]:
time_polys = {time: [] for time in trip_times}
for _, lot in tqdm(gdf.iterrows()):
    polys = make_iso_polys(projected_graph, lot['nearest_node'], edge_buff=25, node_buff=0, infill=True)
    for time, polygon in polys.items():
        properties = lot.to_dict()
        del properties['geometry']
        time_polys[time].append(dict(poly=polygon.simplify(1), properties=properties))

0it [00:00, ?it/s]

## Save the isochrones

In [26]:
for time, polys in time_polys.items():
    features = []
    for polygon in polys:
        properties = {'time': time, **polygon['properties']}
        geometry = get_geojson_geometry(polygon['poly'])
        features.append(dict(type='Feature', properties=properties, geometry=geometry))

    # Save geojson
    geojson = { "type": "FeatureCollection", "features": features }
    with open(f'./parking_lot_isochrones_{time}min.geojson', 'w') as f:
        json.dump(geojson, f)

# Save the union of isochrones for each time

In [130]:
features = []
for time, polys in time_polys.items():
    union = shapely.ops.unary_union([poly['poly'] for poly in polys])
    
    # for poly in union.geoms:
    #     feature = {"type": "Feature", "properties": { "time": time }, "geometry": get_geojson_geometry(poly.simplify(1))}
    #     features.append(feature)

    feature = {"type": "Feature", "properties": { "time": time }, "geometry": get_geojson_geometry(union.simplify(1))}
    features.append(feature)

geojson = { "type": "FeatureCollection", "features": features }
with open('parking_lot_isochrone_unions.geojson', 'w') as f:
    json.dump(geojson, f)